<br>

# Requests

Em 13.10.2025 retomei a ideia de obter os dados por meio de `requests`.

<br>

Michel Metran\
Data: XX.XX.2024\
Atualizado em: 13.10.2025\


In [ ]:
#!pip3 install certifi --upgrade
#!pip3 install geopandas==1.1.1

In [ ]:
from urllib.parse import quote, urljoin

import geopandas as gpd
import requests
import urllib3
from bs4 import BeautifulSoup
from paths import input_path, output_path
from urllib3.util import create_urllib3_context

In [ ]:
import ssl
import urllib.request
from urllib.parse import (
    parse_qs,
    parse_qsl,
    quote_plus,
    unquote,
    urlencode,
    urlparse,
    urlsplit,
    urlunparse,
    urlunsplit,
)

import certifi
import lxml
from lxml import etree, html

In [ ]:
# import portaldatransparencia as pt

<br>

---

## Funcionamento do INCRA

O INCRA mantem uma interface bastante simples para seleção do _layer_ desejado no endereço

- https://certificacao.incra.gov.br/csv_shp/export_shp.py


<br>

---

## Certificados


In [ ]:
#certifi.where()

In [ ]:
# # Create an SSL context with certifi's CA bundle
# ssl_context = ssl.create_default_context(cafile=certifi.where())

# # Create an SSL context that does not verify certificates
# ssl_context = ssl._create_unverified_context()

# ssl_context

<br>

---

## Requests

É possível fazer a consulta com o `verify=False`. Mas, é bom evitar...


In [ ]:
url = 'https://certificacao.incra.gov.br/csv_shp/zip/Áreas%20de%20Quilombolas.zip'
r = requests.get(url=url, verify=False)
r

<br>

---

## Funções

Observei que o site do INCRA usa, ainda, TLS 1.2. O python "reclama" disso.


In [ ]:
class CustomSSLContextHTTPAdapter(requests.adapters.HTTPAdapter):
    def __init__(self, ssl_context=None, **kwargs) -> None:
        self.ssl_context = ssl_context
        super().__init__(**kwargs)

    def init_poolmanager(
        self, connections, maxsize, block=False, **kwargs
    ) -> None:
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections,
            maxsize=maxsize,
            block=block,
            ssl_context=self.ssl_context,
        )

In [ ]:
ctx = create_urllib3_context()
ctx.load_default_certs()
ctx.set_ciphers('AES256-GCM-SHA384')

session = requests.session()
session.adapters.pop('https://', None)
session.mount('https://', CustomSSLContextHTTPAdapter(ctx))

<br>

Cria a URL do _layer_


In [ ]:
# Parâmetros
filename = 'Assentamento Reconhecimento.zip'
url_base = 'https://certificacao.incra.gov.br/csv_shp/zip/'

# Contrói a URL
url_lyr = urljoin(base=url_base, url=quote(filename))
url_lyr

In [ ]:
s = session.get(url=url_lyr)
s

<br>

-----

## *Download* em *chuncks*

In [ ]:
# Faz requisição
s = session.get(url=url_lyr, stream=True)

# Define nome do arquivo
output_file = input_path / filename

with open(file=output_file, mode='wb') as f:
    for chunk in s.iter_content(chunk_size=8192):
        if chunk:  # filtra keep-alive chunks
            f.write(chunk)

<br>

------

## Dicionários

In [ ]:
dict_layers = {
    'Imóvel certificado SIGEF Total': {
        'filename': 'Sigef Brasil.zip',
        'dowpdown': 'sigef',
    },
    'Imóvel certificado SIGEF Público': {
        'filename': 'Sigef Público.zip',
        'dowpdown': 'sigef_pub',
    },
    'Imóvel certificado SIGEF Privado': {
        'filename': 'Sigef Privado.zip',
        'dowpdown': 'sigef_pri',
    },
    'Imóvel certificado SNCI Total': {
        'filename': 'Imóvel certificado SNCI Brasil.zip',
        'dowpdown': 'snci',
    },
    'Imóvel certificado SNCI Público': {
        'filename': 'Imóvel certificado SNCI Público.zip',
        'dowpdown': 'snci_pub',
    },
    'Imóvel certificado SNCI Privado': {
        'filename': 'Imóvel certificado SNCI Privado.zip',
        'dowpdown': 'snci_pri',
    },
    'Projetos de Assentamento Total': {
        'filename': 'Assentamento Brasil.zip',
        'dowpdown': 'assent',
    },
    'Projetos de Assentamento Federal': {
        'filename': 'Assentamento Federal.zip',
        'dowpdown': 'assent_f',
    },
    'Projetos de Assentamento Reconhecimento': {
        'filename': 'Assentamento Reconhecimento.zip',
        'dowpdown': 'assent_r',
    },
    'Áreas de Quilombolas': {
        'filename': 'Áreas de Quilombolas.zip',
        'dowpdown': 'quilom',
    },
}

uf = 'SP'
# uf = None
url_base = 'https://certificacao.incra.gov.br/csv_shp/zip/'

# a = 'Sigef%20Brasil.zip'

for k, v in dict_layers.items():
    print('-' * 60)
    print(k)

    # Parâmetro
    filename = v['filename']
    dowpdown = v['dowpdown']

    # ddddd
    if uf is None:
        uf_fix = ''
    else:
        uf_fix = uf

    # Faz o request para atualizar o dado
    s = session.post(
        url='https://certificacao.incra.gov.br/csv_shp/export_shp.py',
        params={'selectshp': dowpdown, 'selectuf': uf_fix},
        stream=True,
    )

    # Estado
    if uf is not None:
        filename = filename.strip()
        filename = filename.removesuffix('.zip')
        filename = f'{filename}_{uf}.zip'

    # print(filename)

    # Contrói a URL
    url_lyr = urljoin(
        base=url_base.strip(),
        url=quote(filename),
    )
    print(url_lyr)

    # Faz requisição
    s = session.get(url=url_lyr, stream=True)

    # Define nome do arquivo
    output_file = input_path / filename

    # Faz o request para obter o dado
    with open(file=output_file, mode='wb') as f:
        for chunk in s.iter_content(chunk_size=8192):
            if chunk:  # filtra keep-alive chunks
                f.write(chunk)

<br>

---

## Update


In [ ]:
# Faz requisição para update
s = session.post(
    url='https://certificacao.incra.gov.br/csv_shp/export_shp.py',
    params={'selectshp': 'quilom', 'selectuf': 'AP'},
    stream=True,
)

s.status_code

In [ ]:
r.content

<br>

------------

## Geopandas

In [ ]:
gdf = gpd.read_file(filename=input_path / 'Sigef Público_SP.zip')

In [ ]:
gdf.crs

In [ ]:
gdf.info()
gdf.head()

<br>

------------

## Outros

In [ ]:
s = session.get(url=url, allow_redirects=True)
s.content

In [ ]:
#page = urllib.request.urlopen(url)
page = urllib.request.urlopen(url=url, context=ssl_context)
page.geturl()

In [ ]:
soup = BeautifulSoup(page, 'html.parser')
print(soup.prettify())

In [ ]:
BeautifulSoup(s)

In [ ]:
content = urllib.request.urlopen(url=url, context=ssl_context).read()
tree = html.fromstring(content)
tree

In [ ]:
content

In [ ]:
list_divs = tree.xpath("//select[@id='selectshp']")
list_divs = tree.xpath("//form[@class='form-horizontal form_1']")

list_divs

In [ ]:
r.content

In [ ]:
'https://certificacao.incra.gov.br/csv_shp/zip/%C3%81reas%20de%20Quilombolas_AP.zip'

In [ ]:
Path('https://certificacao.incra.gov.br/csv_shp/zip')

In [ ]:


# Configure o seu token
pt.config(token='SEU_TOKEN')

# Exemplo: buscando dados sobre despesas
despesas = pt.despesas.get_despesas(ano=2024, mes=5)

# Faça algo com os dados, como imprimir ou salvar em um arquivo
for despesa in despesas:
    print(despesa)